In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-water-quality-data/water_dataX.csv


In [4]:
import os 
print(os.listdir("../input/indian-water-quality-data"))

['water_dataX.csv']


In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [16]:
# Load and preprocess the data
df = pd.read_csv("../input/indian-water-quality-data/water_dataX.csv", encoding='unicode_escape')
df = df.iloc[0:1900, :]
df = df.rename(columns={"D.O. (mg/l)": "DO", "CONDUCTIVITY (µmhos/cm)": "Conductivity", "B.O.D. (mg/l)": "BOD", 
                        "NITRATENAN N+ NITRITENANN (mg/l)": "NI", "FECAL COLIFORM (MPN/100ml)": "Fec_col", 
                        "TOTAL COLIFORM (MPN/100ml)Mean": "Tot_col"})
df

,STATION CODE,LOCATIONS,STATE,Temp,DO,PH,Conductivity,BOD,NI,Fec_col,Tot_col,year
0,1393,"DAMANGANGA AT D/S OF MADHUBAN, DAMAN",DAMAN & DIU,30.6,6.7,7.5,203,NAN,0.1,11,27,2014
1,1399,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOI...,GOA,29.8,5.7,7.2,189,2,0.2,4953,8391,2014
2,1475,ZUARI AT PANCHAWADI,GOA,29.5,6.3,6.9,179,1.7,0.1,3243,5330,2014
3,3181,RIVER ZUARI AT BORIM BRIDGE,GOA,29.7,5.8,6.9,64,3.8,0.5,5382,8443,2014
4,3182,RIVER ZUARI AT MARCAIM JETTY,GOA,29.5,5.8,7.3,83,1.9,0.4,3428,5500,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
1895,NAN,"TAMBIRAPARANI AT RAIL BDG. NR. AMBASAMUDAM, TA...",NAN,27,7.4,7,88.5,0.977,0.186,27,105,2005
1896,NAN,"TAMBIRAPARANI AT ARUMUGANERI, TAMILNADU",NAN,NAN,6.6,7.81,603.2,2.675,0.263,40,191,2005
1897,NAN,"PALAR AT VANIYAMBADI WATER SUPPLY HEAD WORK, T...",NAN,28,6.6,7.49,571.5,2.091,0.256,151,273,2005
1898,NAN,"GUMTI AT U/S SOUTH TRIPURA,TRIPURA",NAN,28,5.4,7.16,75.8,2.092,NAN,404,513,2005


In [14]:
# Dropping unwanted features
df_num = df.drop(["STATION CODE", "NI", "LOCATIONS", "STATE", "year", "BOD", "Fec_col", "Tot_col"], axis=1)
df_num

,Temp,DO,PH,Conductivity
0,30.6,6.7,7.50,203.0
1,29.8,5.7,7.20,189.0
2,29.5,6.3,6.90,179.0
3,29.7,5.8,6.90,64.0
4,29.5,5.8,7.30,83.0
...,...,...,...,...
1895,27.0,7.4,7.00,88.5
1896,NaN,6.6,7.81,603.2
1897,28.0,6.6,7.49,571.5
1898,28.0,5.4,7.16,75.8


In [9]:
num_col = df.shape[1]
for index in range(num_col):
    col_name = df.iloc[:, index].name
    df[col_name] = pd.to_numeric(df[col_name], errors="coerce")

df.dtypes

STATION CODE    float64
LOCATIONS       float64
STATE           float64
Temp            float64
DO              float64
PH              float64
Conductivity    float64
BOD             float64
NI              float64
Fec_col         float64
Tot_col         float64
year              int64
dtype: object

In [10]:
# Convert all columns to numeric, coerce errors to NaN
df_num = df_num.apply(pd.to_numeric, errors='coerce')

In [ ]:
def convert_to_nan(df):
    n_col = df.shape[1]
    for index in range(n_col):
        df.iloc[:, index]  = df.iloc[:, index].replace("NAN", np.nan)
    return df

df = convert_to_nan(df)
df.isnull().sum().sort_values()

In [13]:
# Convert all columns to numeric, coerce errors to NaN
df_num = df_num.apply(pd.to_numeric, errors='coerce')

# Function to replace "NAN" strings with np.nan
def convert_to_nan(df):
    n_col = df.shape[1]
    for index in range(n_col):
        df.iloc[:, index] = df.iloc[:, index].replace("NAN", np.nan)
    return df

df_num = convert_to_nan(df_num)

,Temp,DO,PH,Conductivity
0,30.6,6.7,7.50,203.0
1,29.8,5.7,7.20,189.0
2,29.5,6.3,6.90,179.0
3,29.7,5.8,6.90,64.0
4,29.5,5.8,7.30,83.0
...,...,...,...,...
1895,27.0,7.4,7.00,88.5
1896,NaN,6.6,7.81,603.2
1897,28.0,6.6,7.49,571.5
1898,28.0,5.4,7.16,75.8


In [ ]:
# Impute missing values with median
imputer = SimpleImputer(strategy="median")
df_num = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns)


In [ ]:
# Detecting and removing outliers using Z-Score
def detect_outliers_zscore(df, threshold=3):
    z_scores = zscore(df)
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < threshold).all(axis=1)
    return df[filtered_entries]

df_num = detect_outliers_zscore(df_num)


In [ ]:
# Plot histograms
def plot_histograms(df):
    for col in df.columns:
        fig, ax = plt.subplots(1, 1, figsize=(7, 5))
        sns.histplot(df[col], kde=True, ax=ax)
        ax.set_title(f'Distribution of {col}')
        plt.show()

plot_histograms(df_num)


In [ ]:
# KDE plots
def plot_kde(df):
    for col in df.columns:
        fig, ax = plt.subplots(1, 1, figsize=(7, 5))
        sns.kdeplot(data=df, x=col, ax=ax)
        ax.set_title(f'KDE Plot of {col}')
        plt.show()

plot_kde(df_num)

In [ ]:
# Correlation heatmap
corr = df_num.corr()
fig, ax = plt.subplots(figsize=(13, 9))
sns.heatmap(corr, annot=True, cmap='magma', ax=ax)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Pairplot
sns.pairplot(df_num)
plt.show()

In [ ]:
# Boxplot
def plot_boxplots(df):
    for col in df.columns:
        fig, ax = plt.subplots(1, 1, figsize=(7, 5))
        sns.boxplot(data=df, x=col, ax=ax)
        ax.set_title(f'Boxplot of {col}')
        plt.show()

plot_boxplots(df_num)

In [ ]:
# Scatter plot matrix
pd.plotting.scatter_matrix(df_num, figsize=(12, 12))
plt.show()

In [ ]:
# Redefining df_num based on the previously processed dataframe
df_num = df.drop(["STATION CODE", "NI", "LOCATIONS", "STATE", "year", "BOD", "Fec_col", "Tot_col"], axis=1)
df_num = df_num.apply(pd.to_numeric, errors='coerce')
df_num = convert_to_nan(df_num)
df_num.head()


In [ ]:
# Replacing NULL values with median of column
imputer = SimpleImputer(strategy="median")
df_num = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns)
df_num.head()

In [ ]:
# Compute Z-Score
df_num = detect_outliers_zscore(df_num)
df_num.head()

In [ ]:
# Temp has nothing to do with WQI, so dropping
df_num.drop(["Temp"], axis=1, inplace=True)

In [ ]:
# Weight Vector (wi)
wi = np.array([0.2213, 0.2604, 0.0022])
wi

In [ ]:
# Standard values of parameters (si)
si = np.array([10, 8.5, 1000])
si

In [ ]:
# Ideal values of parameters (vIdeal)
vIdeal = np.array([14.6, 7, 0])

def calc_wqi(sample): 
    wqi_sample = 0
    num_col = 3
    for index in range(num_col):
        v_index = sample[index]  # Observed value of sample at index
        v_index_ideal = vIdeal[index]  # Ideal value of observed value
        w_index = wi[index]  # Weight of corresponding parameter of observed value
        std_index = si[index]  # Standard value recommended for observed value
        q_index = (v_index - v_index_ideal) / (std_index - v_index_ideal)
        q_index = q_index * 100  # Final qi value of observed value
        wqi_sample += q_index * w_index
    return wqi_sample
def calc_wqi_for_df(df):
    wqi_arr = []
    for index in range(df.shape[0]):
        index_row = df.iloc[index, :]
        wqi_row = calc_wqi(index_row)
        wqi_arr.append(wqi_row)
    return wqi_arr

wqi_arr = calc_wqi_for_df(df_num)
wqi_arr = np.array(wqi_arr).reshape(-1, 1)
# wqi_df = pd.DataFrame(wqi_arr, columns=["WQI"])
wqi_arr

In [ ]:
# Combining dataframe of WQI and dataframe of attributes
df_wqi = pd.concat([df_num.reset_index(drop=True), pd.DataFrame(wqi_arr, columns=["WQI"])], axis=1)
df_wqi.tail()

In [ ]:
#  Removing the samples with negative WQI
df_wqi = df_wqi[df_wqi["WQI"] >= 0]
df_wqi.head()

In [ ]:
# WQI classification
df_wqi["WQI clf"] = df_wqi["WQI"].apply(lambda x: 4 if x <= 25 else 3 if 26 <= x <= 50 else 2 if 51 <= x <= 75 else 1 if 76 <= x <= 100 else 0)

In [ ]:
features = list(df_wqi.columns)[:]
data_cluster = df_wqi['WQI clf']
data_cluster.describe()

 WQI Range                   Classification

     Less than 25                  Excellent(3)
        26–50                         Good(2)
        51-75                         Poor(1)
    Greater than 75                 Very Poor(0)

In [ ]:
df_wqi["WQI clf"] = df_wqi["WQI"].apply(lambda x: (4 if (x <= 25)
                                        else(3 if (26<=x<=50)
                                        else(2 if (51<=x<=75)
                                        else(1 if (76<=x<=100)
                                        else 0)))))
df_wqi.head(10)

In [ ]:
# Correlation heatmap
corr = df_wqi[["DO", "PH", "Conductivity", "WQI"]].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(13, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .7})
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

In [ ]:
# KMeans clustering and Elbow method
features = list(df_wqi.columns)[:3]
data_f = df_wqi[features]

# Ensure data_f is not empty before proceeding
if data_f.empty:
    raise ValueError("No data available in data_f. Please check the data preprocessing steps.")

sse = []
list_k = list(range(1, 10))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(data_f)
    sse.append(km.inertia_)
    
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance')
plt.title('Elbow Method for Optimal k')
plt.show()

In [ ]:
# Logistic Regression
Y = df_wqi['WQI clf']
X = df_wqi[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=30)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# Clustering with Silhouette Score
kmeans = KMeans(n_clusters=3)
kmeans.fit(X_scaled)
labels = kmeans.labels_
silhouette_score(X_scaled, labels, metric='euclidean') # silhouette

In [ ]:
# Histogram for WQI
plt.figure(figsize=(10, 6))
sns.histplot(df_wqi['WQI'], bins=30, kde=True)
plt.title('Histogram of Water Quality Index (WQI)')
plt.xlabel('WQI')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Count plot for WQI classification
plt.figure(figsize=(10, 6))
sns.countplot(x=df_wqi['WQI clf'])
plt.title('Count Plot of WQI Classification')
plt.xlabel('WQI Classification')
plt.ylabel('Count')
plt.show()


In [ ]:
# Heatmap of the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
plt.title('Confusion Matrix Heatmap')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
# Bar plot of WQI classification counts
wqi_class_counts = df_wqi['WQI clf'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
sns.barplot(x=wqi_class_counts.index, y=wqi_class_counts.values, palette='viridis')
plt.title('Bar Plot of WQI Classification Counts')
plt.xlabel('WQI Classification')
plt.ylabel('Count')
plt.show()


In [ ]:
# Box plot for various features
plt.figure(figsize=(15, 8))
sns.boxplot(data=df_num)
plt.title('Box Plot for Various Features')
plt.xticks(rotation=90)
plt.show()


In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 8))
corr_matrix = df_num.corr()
sns.heatmap(corr_matrix, annot=True, cmap='magma', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()
